In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("recommend").getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
data = spark.read.csv("/home/shoaibalauddin/spark-2.4.4-bin-hadoop2.7/python/pyspark_notebooks/dataset/recommend/movielens_ratings.csv", inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [9]:
data.show(), data.count()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



(None, 1501)

In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [11]:
train_data, test_data = data.randomSplit([0.8,0.2])

In [12]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol='rating')

In [13]:
model = als.fit(train_data)

In [14]:
predictions = model.transform(test_data)

In [15]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    26| -0.6401658|
|     31|   1.0|    13|    2.08215|
|     31|   3.0|     7| 0.58454955|
|     31|   3.0|    14|  1.1053808|
|     31|   1.0|    18|  0.9574346|
|     85|   1.0|     5| 0.96373326|
|     85|   3.0|    21|  1.9276595|
|     85|   1.0|     2|  4.2325606|
|     65|   1.0|    28|  1.2167697|
|     65|   2.0|     3| -1.8824242|
|     65|   2.0|    15| 0.72068304|
|     65|   5.0|    23|0.044882715|
|     53|   1.0|     6|  1.9148036|
|     53|   5.0|    21|  3.8619025|
|     53|   3.0|    14|  3.3792882|
|     78|   1.0|    13| 0.91000825|
|     78|   1.0|    24|  0.7730464|
|     78|   1.0|    11|  0.6872341|
|     34|   1.0|    15|   2.479125|
|     34|   3.0|    25|  2.8325725|
+-------+------+------+-----------+
only showing top 20 rows



In [16]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [17]:
rmse = evaluator.evaluate(predictions)

In [18]:
print("RMSE :", rmse)

RMSE : 1.8571418024597104


In [31]:
single_user = test_data.filter(test_data["userId"] == 11).select(["movieId", "userId", "rating"])

In [32]:
type(single_user)

pyspark.sql.dataframe.DataFrame

In [33]:
single_user.show()

+-------+------+------+
|movieId|userId|rating|
+-------+------+------+
|      6|    11|   2.0|
|     10|    11|   1.0|
|     11|    11|   1.0|
|     12|    11|   1.0|
|     13|    11|   4.0|
|     16|    11|   1.0|
|     19|    11|   4.0|
|     21|    11|   1.0|
|     25|    11|   1.0|
|     27|    11|   5.0|
|     36|    11|   2.0|
|     38|    11|   4.0|
|     59|    11|   1.0|
|     64|    11|   1.0|
|     76|    11|   1.0|
|     78|    11|   1.0|
|     80|    11|   3.0|
|     82|    11|   1.0|
|     86|    11|   1.0|
|     97|    11|   3.0|
+-------+------+------+



In [36]:
recommendations = model.transform(single_user)

In [37]:
recommendations.show()

+-------+------+------+------------+
|movieId|userId|rating|  prediction|
+-------+------+------+------------+
|     78|    11|   1.0|   0.6872341|
|     76|    11|   1.0|  0.33660057|
|     27|    11|   5.0|    4.273672|
|     12|    11|   1.0|   1.0750368|
|     13|    11|   4.0|   1.9275416|
|      6|    11|   2.0|   2.4323409|
|     16|    11|   1.0|   1.2606089|
|     86|    11|   1.0|-0.021483898|
|     19|    11|   4.0|   1.2914331|
|     64|    11|   1.0|    0.917824|
|     59|    11|   1.0|   1.9290086|
|     97|    11|   3.0|    1.133211|
|     10|    11|   1.0|  0.61268187|
|     38|    11|   4.0| -0.46780372|
|     82|    11|   1.0|   0.9075024|
|     80|    11|   3.0|   0.9119985|
|     25|    11|   1.0|  0.90820533|
|     21|    11|   1.0| -0.99104637|
|     11|    11|   1.0| 0.058502376|
|     36|    11|   2.0|   1.2019975|
+-------+------+------+------------+



In [38]:
recommendations.orderBy("prediction", ascending=False).show()

+-------+------+------+------------+
|movieId|userId|rating|  prediction|
+-------+------+------+------------+
|     27|    11|   5.0|    4.273672|
|      6|    11|   2.0|   2.4323409|
|     59|    11|   1.0|   1.9290086|
|     13|    11|   4.0|   1.9275416|
|     19|    11|   4.0|   1.2914331|
|     16|    11|   1.0|   1.2606089|
|     36|    11|   2.0|   1.2019975|
|     97|    11|   3.0|    1.133211|
|     12|    11|   1.0|   1.0750368|
|     64|    11|   1.0|    0.917824|
|     80|    11|   3.0|   0.9119985|
|     25|    11|   1.0|  0.90820533|
|     82|    11|   1.0|   0.9075024|
|     78|    11|   1.0|   0.6872341|
|     10|    11|   1.0|  0.61268187|
|     76|    11|   1.0|  0.33660057|
|     11|    11|   1.0| 0.058502376|
|     86|    11|   1.0|-0.021483898|
|     38|    11|   4.0| -0.46780372|
|     21|    11|   1.0| -0.99104637|
+-------+------+------+------------+

